In [2]:
n_steps_in = 36
# n_steps_out = 24
n_steps_out = 5
# n_steps_out = 48
# n_steps_out = 1

features = ['SWTP Total Influent Flow', 'Fire Rainfall (in)', 'Bingham Rainfall (in)', 
            "Ozark Aquifer Depth to Water Level (ft)", "James Gauge Height (ft)", 'HourlyStationPressure']


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, LSTM, Activation
import keras_tuner as kt
from sklearn.metrics import mean_squared_error
from keras_tuner.tuners import BayesianOptimization
import os

def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

def sliding_window(X, y, n_test, slide):
    split_point = X.shape[0] - n_test + slide
    train_X , train_y = X[:split_point, :] , y[:split_point, :]
    test_X , test_y = X[split_point:, :] , y[split_point:, :]
    return train_X, train_y, test_X, test_y

def getValidationData(filename, features, n_steps_in):
    # getting data
    dataset = pd.read_csv(filename, usecols=features)
    arr = np.array(dataset["SWTP Total Influent Flow"])
    dataset["Target"] = arr         # adding another influent flow feature so that past values can be used to predict future values
    values = dataset.values
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(values)

    X, y = split_sequences(scaled, n_steps_in, n_steps_out)
    train_X, train_y, validate_X, validate_y = sliding_window(X, y, 8760, 0)    # 8760 means last year of data, for validation
    return validate_X, validate_y

def invNormalize(arr, minimum, maximum):
    return (maximum - minimum) * arr + minimum

def mseForecast(y, y_pred):
    totalMSE = 0
    for i in range(y.shape[0]):
        totalMSE += mean_squared_error(y[i], y_pred[i])
    avgMSE = totalMSE / y.shape[0]
    return avgMSE

def predict(model, test_X, test_y, fullData = False):
    # to be able to inverse scale predictions
    df = pd.read_csv("Train and Test Data.csv", usecols=["SWTP Total Influent Flow"])
    if fullData:
        df = pd.read_csv("Imputed Data.csv", usecols=["SWTP Total Influent Flow"])
    arr = np.array(df["SWTP Total Influent Flow"])
    maximum = np.max(arr)
    minimum = np.min(arr)

    #predictions and rescaling to [min, max]
    y_pred = model.predict(test_X)
    y_pred_inv = np.array([invNormalize(x, minimum, maximum) for x in y_pred])
    test_y_inv = np.array([invNormalize(x, minimum, maximum) for x in test_y])
    print("y_pred_inv:",y_pred_inv.shape)
    print("test_y_inv:",y_pred_inv.shape)
    
    return y_pred_inv, test_y_inv

def saveValidationResults(path, testMSE, validationMSE, n_epochs, batch_size, hours=36, linexLossA = 3):
    txt = "n_steps_in = " + str(hours)
    txt += "\nepochs = " + str(n_epochs)
    txt += "\nbatch = " + str(batch_size)
    txt += "\nlinex loss a = " + str(linexLossA)
    txt += "\nAvg Test MSE: " + str(round(testMSE, 4))
    txt += "\nAvg Validation MSE: " + str(round(validationMSE, 4))
    txt += "\n\nForm:\nLSTM\nLSTM\nDense(24)\nActivation('linear')"
    with open(path + "\\validation results.txt", 'w') as f:
        f.write(txt)


def sign_ae(x, y):
    sign_x = tf.math.sign(x)
    sign_y = tf.math.sign(y)
    delta = x - y
    return sign_x * sign_y * tf.math.abs(delta)
    
def linex_loss(delta, a=3., b=3.):
    if a != 0 and b > 0:
        # loss = 1/(a*a) * (tf.exp(a * delta) - a * delta - 1)
        loss = b * (tf.exp(a * delta) - a * delta - 1)
        return loss
    else:
        raise ValueError("linex_loss error with a or b")
          
def linex_loss_val(y_true, y_pred):
    delta = sign_ae(y_true, y_pred)
    res = linex_loss(delta)
    return tf.math.reduce_mean(res)


class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        neuronTuning = hp.Int('units', min_value = 200, max_value = 350, step = 8)
        learningRateTuning = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4, 1e-5])
        dropoutRateTuning = hp.Choice('rate', values = [0.0, 0.1, 0.2, 0.3])

        model = keras.Sequential()
        model.add(LSTM(units = neuronTuning, dropout = dropoutRateTuning, 
               activation = 'tanh', input_shape = (n_steps_in, len(features)), return_sequences = True))
        model.add(LSTM(units = neuronTuning, dropout = dropoutRateTuning))
        model.add(Dense(24))
        model.add(Activation('linear'))
        model.compile(loss = 'mse', metrics = 'mse', optimizer = keras.optimizers.Adam(learningRateTuning))
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,  
                         batch_size = hp.Choice("batch_size", [2, 4, 8, 12, 16, 32, 64]), 
                         **kwargs)

def buildModel():
    neurons = 135
    learningRate = 1e-4
    # dropoutRate = 0.2
    dropoutRate = 0

    model = keras.Sequential()
    model.add(LSTM(units = neurons, dropout = dropoutRate, 
            activation = 'tanh', input_shape = (n_steps_in, len(features)), return_sequences = True))
    model.add(LSTM(units = neurons, dropout = dropoutRate))
    model.add(Dense(n_steps_out))
    model.add(Activation('linear'))
    model.compile(loss = linex_loss_val, metrics = linex_loss_val, optimizer = keras.optimizers.Adam(learningRate))
    # model.compile(loss = 'mse', metrics = 'mse', optimizer = keras.optimizers.Adam(learningRate))
    return model


In [5]:
n_epochs = 5
path = "C:\\Users\\natha\\Desktop\\Undergrad\\Spring2022\\MTH 596 PIC Math\\Project - Group 2\\Project\\Forecasting"
path += "\\loss\\linex loss 1 hour"
modelPath = path + "\\model"

dataset = pd.read_csv("Train and Test Data.csv", usecols=features)
arr = np.array(dataset["SWTP Total Influent Flow"])
dataset["Target"] = arr
values = dataset.values
scaler = MinMaxScaler()                     # automatically [0, 1]
scaled = scaler.fit_transform(values)
X, y = split_sequences(scaled, n_steps_in, n_steps_out)
train_X, train_y, test_X, test_y = sliding_window(X, y, 365*24, 0)

model = buildModel()
hist = model.fit(train_X, train_y, epochs = n_epochs, validation_data = (test_X, test_y), 
                    validation_split = 0.2, verbose = 1, batch_size = 4)
model.save(modelPath)
# model = keras.models.load_model(modelPath, custom_objects={'linex_loss_val': linex_loss_val})

pred_y_inv, test_y_inv = predict(model, test_X, test_y)
avgTestMSE = mseForecast(test_y_inv, pred_y_inv)
validate_X, validate_y = getValidationData("Imputed Data.csv", features, n_steps_in)
y_pred_inv, y_validate_inv = predict(model, validate_X, validate_y, True)
validationAvgMSE = mseForecast(y_validate_inv, y_pred_inv)
print(validationAvgMSE)

saveValidationResults(path, avgTestMSE, validationAvgMSE, n_epochs, 4, n_steps_in)

Epoch 1/5
5341/5341 [==============================] - 117s 21ms/step - loss: 0.0183 - linex_loss_val: 0.0183 - val_loss: 0.0075 - val_linex_loss_val: 0.0075
Epoch 2/5
5341/5341 [==============================] - 107s 20ms/step - loss: 0.0028 - linex_loss_val: 0.0028 - val_loss: 8.7994e-04 - val_linex_loss_val: 8.7963e-04
Epoch 3/5
5341/5341 [==============================] - 108s 20ms/step - loss: 3.6860e-04 - linex_loss_val: 3.6860e-04 - val_loss: 3.0227e-04 - val_linex_loss_val: 3.0235e-04
Epoch 4/5
5341/5341 [==============================] - 111s 21ms/step - loss: 1.3420e-04 - linex_loss_val: 1.3420e-04 - val_loss: 1.3036e-04 - val_linex_loss_val: 1.3035e-04
Epoch 5/5
5341/5341 [==============================] - 108s 20ms/step - loss: 7.7698e-05 - linex_loss_val: 7.7698e-05 - val_loss: 1.5994e-05 - val_linex_loss_val: 1.5997e-05


INFO:tensorflow:Assets written to: C:\Users\natha\Desktop\Undergrad\Spring2022\MTH 596 PIC Math\Project - Group 2\Project\Forecasting\loss\linex loss 1 hour\model\assets


INFO:tensorflow:Assets written to: C:\Users\natha\Desktop\Undergrad\Spring2022\MTH 596 PIC Math\Project - Group 2\Project\Forecasting\loss\linex loss 1 hour\model\assets


y_pred_inv: (8760, 1)
test_y_inv: (8760, 1)
y_pred_inv: (8760, 1)
test_y_inv: (8760, 1)
0.026083060895557397


In [ ]:
epochList = [5]
lastAttempt = 3
attemptList = list(range(lastAttempt+1, len(epochList) + lastAttempt+1))

for attempt, n_epochs in zip(attemptList, epochList):
    # getting data and splitting it up
    dataset = pd.read_csv("Train and Test Data.csv", usecols=features)
    arr = np.array(dataset["SWTP Total Influent Flow"])
    dataset["Target"] = arr
    values = dataset.values
    scaler = MinMaxScaler()     # automatically [0, 1]
    scaled = scaler.fit_transform(values)
    X, y = split_sequences(scaled, n_steps_in, n_steps_out)
    train_X, train_y, test_X, test_y = sliding_window(X, y, 365*24, 0)


    # getting strings set up
    path = "C:\\Users\\natha\\Desktop\\Undergrad\\Spring2022\\MTH 596 PIC Math\\Project - Group 2\\Project\\Forecasting"
    path += "\\feature tuning\\rainfall subset\\keras_tuner_attempt"
    path += str(attempt)
    modelPath = path + "\\model"
    project_title = "keras_tuner_attempt" + str(attempt)
    dirPath = 'C:/Users/natha/Desktop/Undergrad/Spring2022/MTH 596 PIC Math/Project - Group 2/Project/Forecasting/feature tuning/rainfall subset'

    # creating hyperparameter tuner and tuning with it
    bayesian_opt_tuner = BayesianOptimization(
            MyHyperModel(),
            objective='mse',
            max_trials=3,
            executions_per_trial=1,
            directory=os.path.normpath(dirPath),
            project_name=project_title,
            overwrite=False)                  # switch to false if want to resume from where last left off
            # overwrite=True)
    bayesian_opt_tuner.search(train_X, train_y, epochs=n_epochs,
        validation_data=(test_X, test_y),
        verbose=1,
        callbacks = [keras.callbacks.EarlyStopping('val_loss', patience=2)])        # this will stop if 3 epochs have gone by with no improvement
    bayes_opt_model_best_model = bayesian_opt_tuner.get_best_models(num_models=1)
    model = bayes_opt_model_best_model[0]
    model.save(modelPath)

    pred_y_inv, test_y_inv = predict(model, test_X, test_y)
    avgTestMSE = mseForecast(test_y_inv, pred_y_inv)

    validate_X, validate_y = getValidationData("Imputed Data.csv", features, n_steps_in)
    y_pred_inv, y_validate_inv = predict(model, validate_X, validate_y, True)
    validationAvgMSE = mseForecast(y_validate_inv, y_pred_inv)

    saveValidationResults(path, avgTestMSE, validationAvgMSE, n_epochs, n_steps_in)


In [ ]:
epochList = [5, 6, 7, 8, 9]
attempts = [1, 2, 3, 4, 5]
batches = [4, 16, 32, 64]
subAttempts = [1, 2, 3, 4]

for n_epochs, attempt in zip(epochList, attempts):
    for n_batches, subAttempt in zip(batches, subAttempts):

        # getting data and splitting it up
        dataset = pd.read_csv("Train and Test Data.csv", usecols=features)
        arr = np.array(dataset["SWTP Total Influent Flow"])
        dataset["Target"] = arr
        values = dataset.values
        scaler = MinMaxScaler()     # automatically [0, 1]
        scaled = scaler.fit_transform(values)
        X, y = split_sequences(scaled, n_steps_in, n_steps_out)
        train_X, train_y, test_X, test_y = sliding_window(X, y, 365*24, 0)


        # getting strings set up
        path = "C:\\Users\\natha\\Desktop\\Undergrad\\Spring2022\\MTH 596 PIC Math\\Project - Group 2\\Project\\Forecasting"
        path += "\\loss\\linex loss " + str(attempt) + "." + str(subAttempt)
        modelPath = path + "\\model"
        project_title = "linex loss " + str(attempt) + "." + str(subAttempt)
        dirPath = 'C:/Users/natha/Desktop/Undergrad/Spring2022/MTH 596 PIC Math/Project - Group 2/Project/Forecasting/loss'

        # creating hyperparameter tuner and tuning with it
        model = buildModel()
        hist = model.fit(train_X, train_y, epochs = n_epochs, validation_data = (test_X, test_y), 
                         validation_split = 0.2, verbose = 1, batch_size = n_batches)
        model.save(modelPath)

        pred_y_inv, test_y_inv = predict(model, test_X, test_y)
        avgTestMSE = mseForecast(test_y_inv, pred_y_inv)

        validate_X, validate_y = getValidationData("Imputed Data.csv", features, n_steps_in)
        y_pred_inv, y_validate_inv = predict(model, validate_X, validate_y, True)
        validationAvgMSE = mseForecast(y_validate_inv, y_pred_inv)
        print(validationAvgMSE)

        saveValidationResults(path, avgTestMSE, validationAvgMSE, n_epochs, n_batches, n_steps_in)


In [ ]:
model.summary()